In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re
import openpyxl
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
columns = ["comment"]
df = pd.read_excel(r'C:\Users\Hp\TUGAS AKHIR 2\YOUTUBE_TELKOMSEL\data_mentah\youtube_telkomsel_4.xlsx', usecols=columns)
df.head(2000)

,comment
0,Persi selalu terdepan
1,Press F to TELKOMSEL
2,Juara pisan 😂
3,Mantap
4,Kompak selalu tanpa batas
...,...
123,Kalau di tempat saya yg jadi masalah bukan car...
124,Woy sinyal lag
125,First
126,Jringn tai


In [3]:
df.comment.duplicated().sum()

42

In [4]:
df.drop_duplicates(subset="comment", keep=False, inplace=True)
df.head(2000)

,comment
0,Persi selalu terdepan
1,Press F to TELKOMSEL
2,Juara pisan 😂
3,Mantap
4,Kompak selalu tanpa batas
5,Amin semoga aja
34,Salkomsel selalu
36,Salam kenal bang dari wakatobi
37,Bangle emang Telkom ini setiap main pasti lag....
41,sinyal nge lag parah buat game. padahal unlimi...


In [5]:
#case folding
df['comment'] = df['comment'].str.lower()
df.head(2000)

,comment
0,persi selalu terdepan
1,press f to telkomsel
2,juara pisan 😂
3,mantap
4,kompak selalu tanpa batas
5,amin semoga aja
34,salkomsel selalu
36,salam kenal bang dari wakatobi
37,bangle emang telkom ini setiap main pasti lag....
41,sinyal nge lag parah buat game. padahal unlimi...


In [6]:
#cleaning text
import nltk
import string
import re
nltk.download('punkt')

#import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#proses cleaning text
def remove_special_tweet(text):
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", "", str(text)).split())
    text = re.sub(r'[^a-zA-Z0-9]', ' ', str(text))
    text = re.sub(r'\b\w{1,2}\b', ' ', str(text))
    text = re.sub(r'\s\s+',' ',  str(text))
    text = re.sub(r'\d+', ' ', str(text))
    text = re.sub(r'\b[a-zA-Z]\b', ' ', str(text))
    text = re.sub(r'/(?:https?|ftp):\/\/[\n\S]+/g, ''', ' ', str(text))
    text = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', str(text))
    text = re.sub(r'[0]', r' ', str(text))
    return text
    return df
df['cleaning_comment'] = df['comment'].apply(remove_special_tweet)
df.head(2000)

,comment,cleaning_comment
0,persi selalu terdepan,persi selalu terdepan
1,press f to telkomsel,press telkomsel
2,juara pisan 😂,juara pisan
3,mantap,mantap
4,kompak selalu tanpa batas,kompak selalu tanpa batas
5,amin semoga aja,amin semoga aja
34,salkomsel selalu,salkomsel selalu
36,salam kenal bang dari wakatobi,salam kenal bang dari wakatobi
37,bangle emang telkom ini setiap main pasti lag....,bangle emang telkom ini setiap main pasti lag ...
41,sinyal nge lag parah buat game. padahal unlimi...,sinyal nge lag parah buat game padahal unlimit...


In [8]:
#proses tokenisasi
def tokenisasi(text):
    text = re.split('\W+', str(text))
    return text
    return df
df['tokenisasi'] = df['cleaning_comment'].apply(tokenisasi)
df.head(2000)

,comment,cleaning_comment,tokenisasi
0,persi selalu terdepan,persi selalu terdepan,"[persi, selalu, terdepan]"
1,press f to telkomsel,press telkomsel,"[press, telkomsel]"
2,juara pisan 😂,juara pisan,"[juara, pisan, ]"
3,mantap,mantap,[mantap]
4,kompak selalu tanpa batas,kompak selalu tanpa batas,"[kompak, selalu, tanpa, batas]"
5,amin semoga aja,amin semoga aja,"[amin, semoga, aja]"
34,salkomsel selalu,salkomsel selalu,"[salkomsel, selalu]"
36,salam kenal bang dari wakatobi,salam kenal bang dari wakatobi,"[salam, kenal, bang, dari, wakatobi]"
37,bangle emang telkom ini setiap main pasti lag....,bangle emang telkom ini setiap main pasti lag ...,"[bangle, emang, telkom, ini, setiap, main, pas..."
41,sinyal nge lag parah buat game. padahal unlimi...,sinyal nge lag parah buat game padahal unlimit...,"[sinyal, nge, lag, parah, buat, game, padahal,..."


In [10]:
#import nltk stopword
from nltk.corpus import stopwords

#proses stopword
stopword = nltk.corpus.stopwords.words('indonesian')
stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo','kalo', 'amp', 'biar', 'bikin', 'bilang','gak', 'ga', 'krn', 'nya', 
                 'nih', 'sih','si', 'tau', 'tdk', 'tuh', 'tai', 'juga', 'utk', 'ya','jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nyg', 'hehe',
                 'pen', 'u', 'nan', 'loh', 'rt', 'kudu', 'mks', 'wkwk', 'an', 'lah', 'dah', 'asu', 'mjb', 'haha',
                 'yah', 'lha', 'lah', 'hihi', 'ih', 'anjir', 'bgt', 'pake', 'anj', 'bgst', 'anjir'])
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    return df
df['stopword'] = df['tokenisasi'].apply(remove_stopwords)
df.head(2000)

,comment,cleaning_comment,tokenisasi,stopword
0,persi selalu terdepan,persi selalu terdepan,"[persi, selalu, terdepan]","[persi, terdepan]"
1,press f to telkomsel,press telkomsel,"[press, telkomsel]","[press, telkomsel]"
2,juara pisan 😂,juara pisan,"[juara, pisan, ]","[juara, pisan, ]"
3,mantap,mantap,[mantap],[mantap]
4,kompak selalu tanpa batas,kompak selalu tanpa batas,"[kompak, selalu, tanpa, batas]","[kompak, batas]"
5,amin semoga aja,amin semoga aja,"[amin, semoga, aja]","[amin, semoga]"
34,salkomsel selalu,salkomsel selalu,"[salkomsel, selalu]",[salkomsel]
36,salam kenal bang dari wakatobi,salam kenal bang dari wakatobi,"[salam, kenal, bang, dari, wakatobi]","[salam, kenal, bang, wakatobi]"
37,bangle emang telkom ini setiap main pasti lag....,bangle emang telkom ini setiap main pasti lag ...,"[bangle, emang, telkom, ini, setiap, main, pas...","[bangle, emang, telkom, main, lag, main, game,..."
41,sinyal nge lag parah buat game. padahal unlimi...,sinyal nge lag parah buat game padahal unlimit...,"[sinyal, nge, lag, parah, buat, game, padahal,...","[sinyal, nge, lag, parah, game, unlimitid, swm..."


In [11]:
#stemming
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['stopword']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]
    return df

df['stemming'] = df['stopword'].swifter.apply(get_stemmed_term)
df.head(2000)

172
------------------------
persi : pers
terdepan : depan
press : press
telkomsel : telkomsel
juara : juara
pisan : pis
 : 
mantap : mantap
kompak : kompak
batas : batas
amin : amin
semoga : moga
salkomsel : salkomsel
salam : salam
kenal : kenal
bang : bang
wakatobi : wakatobi
bangle : bangle
emang : emang
telkom : telkom
main : main
lag : lag
game : game
ogah : ogah
gue : gue
kartu : kartu
sinyal : sinyal
nge : nge
parah : parah
unlimitid : unlimitid
swmoga : swmoga
cepet : cepet
bangkrut : bangkrut
udah : udah
kompatibel : kompatibel
perangkat : perangkat
download : download
versi : versi
asli : asli
perbaiki : baik
jaringan : jaring
riau : riau
udh : udh
beil : beil
kouta : kouta
kepakecasw : kepakecasw
benerin : benerin
woy : woy
seminggu : minggu
jawa : jawa
timur : timur
anjim : anjim
banget : banget
jaringannya : jaring
daerah : daerah
jateng : jateng
woyy : woyy
woi : woi
tanggerang : tanggerang
beli : beli
paketan : paket
habis : habis
ratusan : ratus
ribu : ribu
kek : kek
ko

Pandas Apply:   0%|          | 0/46 [00:00<?, ?it/s]

,comment,cleaning_comment,tokenisasi,stopword,stemming
0,persi selalu terdepan,persi selalu terdepan,"[persi, selalu, terdepan]","[persi, terdepan]","[pers, depan]"
1,press f to telkomsel,press telkomsel,"[press, telkomsel]","[press, telkomsel]","[press, telkomsel]"
2,juara pisan 😂,juara pisan,"[juara, pisan, ]","[juara, pisan, ]","[juara, pis, ]"
3,mantap,mantap,[mantap],[mantap],[mantap]
4,kompak selalu tanpa batas,kompak selalu tanpa batas,"[kompak, selalu, tanpa, batas]","[kompak, batas]","[kompak, batas]"
5,amin semoga aja,amin semoga aja,"[amin, semoga, aja]","[amin, semoga]","[amin, moga]"
34,salkomsel selalu,salkomsel selalu,"[salkomsel, selalu]",[salkomsel],[salkomsel]
36,salam kenal bang dari wakatobi,salam kenal bang dari wakatobi,"[salam, kenal, bang, dari, wakatobi]","[salam, kenal, bang, wakatobi]","[salam, kenal, bang, wakatobi]"
37,bangle emang telkom ini setiap main pasti lag....,bangle emang telkom ini setiap main pasti lag ...,"[bangle, emang, telkom, ini, setiap, main, pas...","[bangle, emang, telkom, main, lag, main, game,...","[bangle, emang, telkom, main, lag, main, game,..."
41,sinyal nge lag parah buat game. padahal unlimi...,sinyal nge lag parah buat game padahal unlimit...,"[sinyal, nge, lag, parah, buat, game, padahal,...","[sinyal, nge, lag, parah, game, unlimitid, swm...","[sinyal, nge, lag, parah, game, unlimitid, swm..."


In [12]:
Final = df[['stemming']]
Final = Final.rename(columns={'stemming':'Comments'})

In [13]:
Final

,Comments
0,"[pers, depan]"
1,"[press, telkomsel]"
2,"[juara, pis, ]"
3,[mantap]
4,"[kompak, batas]"
5,"[amin, moga]"
34,[salkomsel]
36,"[salam, kenal, bang, wakatobi]"
37,"[bangle, emang, telkom, main, lag, main, game,..."
41,"[sinyal, nge, lag, parah, game, unlimitid, swm..."


In [14]:
Final.to_excel('databersih_youtube_telkomsel_4.xlsx')